# Using the SDK to pull data and metrics

In [1]:
import datetime
from arthurai.client import ArthurAI
from arthurai.client.apiv2 import MetricType, Stage

## Connect to Arthur and get a reference to a model

In [2]:
config = {
        'access_key': <access_key>,
        'url': 'https://dashboard.arthur.ai/'
    }
connection = ArthurAI(config, version=2)

In [3]:
model = connection.get_model('credit-risk-005')

## Get Data Drift Metrics Over Time Range

In [4]:
cur_time = datetime.datetime.utcnow().isoformat()
three_hours_ago = (datetime.datetime.utcnow() - datetime.timedelta(hours=3)).isoformat()

In [5]:
# Data Drift is computed per attribute in a model
attr = model.get_attributes_for_stage(Stage.ModelPipelineInput)[0]

In [6]:
# Gets metric data for Rollowing Data Drift over three hours with hour granularity
df = model.get_metric_data_as_dataframe(MetricType.RollingDataDrift, start_time=three_hours_ago, 
                                        end_time=cur_time, attribute=attr, granularity='hour')
df

,Data Drift Value,Timestamp,attribute
0,0.027059,2020-04-17T15:00:00,AGE
1,0.879917,2020-04-17T16:00:00,AGE
2,0.403168,2020-04-17T17:00:00,AGE


In [7]:
# gets metric data for Rolling Data Drift over three hours with minute granularity
df = model.get_metric_data_as_dataframe(MetricType.RollingDataDrift, start_time=three_hours_ago, 
                                        end_time=cur_time, attribute=attr, granularity='minute')
df.head(15)

,Data Drift Value,Timestamp,attribute
0,0.069727,2020-04-17T14:47:00,AGE
1,0.273123,2020-04-17T14:48:00,AGE
2,0.529060,2020-04-17T14:49:00,AGE
3,0.637989,2020-04-17T14:50:00,AGE
4,0.544154,2020-04-17T14:51:00,AGE
5,0.730320,2020-04-17T14:52:00,AGE
6,0.760608,2020-04-17T14:53:00,AGE
7,0.160798,2020-04-17T14:54:00,AGE
8,0.244873,2020-04-17T14:55:00,AGE
9,0.638567,2020-04-17T14:56:00,AGE


In [8]:
# gets metric data for Reference Data Drift over three hours with hour granularity
df = model.get_metric_data_as_dataframe(MetricType.ReferenceDataDrift, start_time=three_hours_ago, 
                                        end_time=cur_time, attribute=attr, granularity='hour')
df

,Data Drift Value,Timestamp,attribute
0,0.998703,2020-04-17T15:00:00,AGE
1,0.999972,2020-04-17T16:00:00,AGE
2,0.416364,2020-04-17T17:00:00,AGE


## Get Inferences Over Time Range:
The following sdk function can be used to obtain filtered inference from a specific model in a dataframe format.

In [9]:
# get 3 minutes worth of data
cur_time = datetime.datetime.utcnow().isoformat()
three_minutes_ago = (datetime.datetime.utcnow() - datetime.timedelta(minutes=3)).isoformat()

In [10]:
# filter inferences by age attribute
df = model.get_inferences_as_dataframe(three_minutes_ago, cur_time, 
                                       filters=[{"facet":"AGE","operator":"is equal to","constraint":"26"}])

In [11]:
df.head()

,0,1,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,EDUCATION,...,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SEX,Timestamp,external_id,id
0,0.422499,0.577501,26,71862,73594,71851,68539,49027,48969,2,...,3500,0,3100,1900,2000,1865,2,2020-04-17T17:46:06.432214,257972714,baa52e5e-9730-4d01-af2c-6a72dd4d202e


In [12]:
# Filter for all attributes with the value 7 over a 3 minute window of inferences
df = model.get_inferences_as_dataframe(three_minutes_ago, cur_time, order_by='Timestamp', order_direction='asc',
                                      query_field="7")

In [13]:
df

,0,1,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,EDUCATION,...,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SEX,Timestamp,external_id,id
0,0.927433,0.072567,36,66470,36825,76561,39434,48351,20800,2,...,2000,3000,4200,22000,1200,0,1,2020-04-17T17:43:26.933501,772409132,e219e07e-dad6-4607-b6fb-a6b5ef1d837f
1,0.863974,0.136026,37,187146,175347,181490,169608,163255,168395,1,...,6500,9000,7000,3500,7000,5000,1,2020-04-17T17:43:27.454736,573526820,9b4f380f-c04e-4864-b3ac-f96e9f6925d6
2,0.810633,0.189367,43,2432,261,2745,-626,110,-154,1,...,0,4000,0,1000,0,0,1,2020-04-17T17:43:27.756507,296720995,0e7d381e-d61e-4609-895e-31443fb87438
3,0.799296,0.200704,36,735,735,735,735,735,735,2,...,735,735,735,735,735,735,1,2020-04-17T17:43:28.670649,118351239,5835cbf0-6496-4771-a06e-5c6476736921
4,0.248562,0.751438,36,30809,30015,32928,32100,34557,35357,2,...,0,3400,0,3000,1500,1500,1,2020-04-17T17:43:29.308945,454346484,3edeaa0f-53d4-4b35-a50e-508f87abebc7
5,0.940453,0.059547,37,62053,46888,50783,53204,56946,58394,1,...,75840,5000,5012,10000,5000,10000,1,2020-04-17T17:43:29.932244,538452892,2ef1b0d9-bf76-494d-b3a5-ac9735597767
6,0.863430,0.136570,46,51400,0,0,0,50108,50765,2,...,0,0,0,50108,2363,390,1,2020-04-17T17:43:30.500869,467219929,3a3cf2f7-63f5-4bef-a2f2-b701ecbe5878
7,0.154359,0.845641,46,71532,73643,72083,76603,74964,76370,2,...,3700,0,6147,0,2577,2747,1,2020-04-17T17:43:30.826195,685051879,7b02221a-6443-4efa-b46a-5255eddeac2f
8,0.917996,0.082004,32,120792,87675,122177,120667,47880,49629,1,...,20137,50053,29377,0,30000,984,1,2020-04-17T17:43:31.470318,626582051,b027896f-9598-4a80-aca6-91bb9f5e44ac
9,0.523715,0.476285,30,323,24910,22469,21780,20363,20199,1,...,26703,0,0,1000,800,1000,1,2020-04-17T17:43:32.160265,968934445,1bf05ad1-45d3-41fb-a7d1-c2649ae8e2c6
